In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
#from torch_geometric.loader import DataLoader 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import GCNConv
import pandas as pd
import ast
import torch
from torch.nn import Module, Linear
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, Linear
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.nn import Module, Linear
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import random
import math
import pickle
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import seaborn as sns
import matplotlib
import time
from torch_geometric.nn import GraphSAGE
matplotlib.use('Agg')

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
#from torch_geometric.loader import DataLoader 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch_geometric.nn import GCNConv
import pandas as pd
import ast
import torch
from torch.nn import Module, Linear
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import Module, Linear
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch.nn import Module, Linear
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import random
import math
import pickle
import torch.optim.lr_scheduler as lr_scheduler
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import seaborn as sns
import matplotlib
import time
from torch_geometric.nn import GraphSAGE
import gc
torch.cuda.empty_cache()
gc.collect()
matplotlib.use('Agg')

df = pd.read_csv('/home/scai/msr/aiy227513/scratch/research/superSetAnalysis.csv')
node_ids =[]
node_features = []
req_A=['citationVelocity', 'influentialCitationCount', 'isOpenAccess',
       'isPublisherLicence', 'numCitebBy', 'numCiting','year']
for index, row in df.iterrows():
    paperId = row['paperId']
    node_ids.append(paperId)
    f=[]
    for attribute in req_A:
            if attribute=="isOpenAccess":
             if row[attribute]==True:attribute_value=1
             else:attribute_value=0
            elif attribute=="isPublisherLicence":
              if row[attribute]==True:attribute_value=1
              else:attribute_value=0
            else:
               attribute_value = row[attribute]
            f.append(attribute_value)
    node_features.append(np.array(f).astype(np.float32))
with open("/home/scai/msr/aiy227513/scratch/research/edges_same_ratio.pickle", "rb") as file:
    edges = pickle.load(file)
    file.close()
count=len(edges)
train_size=math.floor((len(edges)*0.8))
train_src=[]
train_dest=[]
train_label=[]
for i in range(train_size):
 if edges[i][2]==1:
    train_src.append(edges[i][0])
    train_dest.append(edges[i][1])
train_size=math.floor((len(edges)*0.8))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with open("/home/scai/msr/aiy227513/scratch/research/data/bert_encode.pkl", "rb") as file:
    embedding = pickle.load(file)
    file.close()
node_features = torch.tensor(node_features, dtype=torch.float)
embedding=torch.tensor(embedding, dtype=torch.float)
feature=torch.cat([node_features,embedding], dim=1).float()
feature =F.normalize(feature,p=2, dim=-1)
class LinkPredictionDataset(Dataset):
    def __init__(self, nodes, node_attributes, edges):
        self.nodes = nodes
        self.node_attributes = node_attributes
        self.edges = edges
        #self.edge_index=edge_index

    def __len__(self):
        return len(self.edges)

    def __getitem__(self, index):
        src, dst,y = self.edges[index]
        # src_node = self.node_attributes[src]
        # dst_node = self.node_attributes[dst]
        #print(src, dst,self.node_attributes,self.y[index])
        return src, dst,self.node_attributes,y
class GCN(nn.Module):
    def __init__(self, in_channels=feature.shape[1], hidden_channels=512, out_channels=128, num_layers=4):
        super().__init__()
        self.conv1 = GraphSAGE(in_channels,out_channels,num_layers)
        #self.conv2 = GraphSAGE(hidden_channels, out_channels,num_layers)
        self.fc1 = Linear(out_channels*2,128)
        #self.fc2=Linear(256,128)
        self.fc3=Linear(128,64)
        self.fc4=Linear(64,16)
        self.fc5=Linear(16,1)
    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        #x = self.conv2(x, edge_index).relu()
        #print(x.shape)
        return x

    def decode(self, z,src,dst):
        #return (z[0][src] * z[0][dst]).sum(dim=-1)
        x=torch.cat([z[0][src],z[0][dst]], dim=-1)
        x=F.leaky_relu(self.fc1(x))
        #x=self.fc2(x).relu()
        x=F.leaky_relu(self.fc3(x))
        x=F.leaky_relu(self.fc4(x))
        x=self.fc5(x)
        return x
edge_index = torch.tensor([train_src,train_dest], dtype=torch.long).to(device)        
model = GCN()
#if torch.cuda.device_count() > 1:model = nn.DataParallel(model)
model.load_state_dict(torch.load('/home/scai/msr/aiy227513/scratch/research/model/model_bert.pth', map_location=device),
                             strict=False)
g_truth=[]
prediction=[]
count=0


/tmp/ipykernel_8231/3591135083.py:77: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  node_features = torch.tensor(node_features, dtype=torch.float)


In [20]:
recommendations = torch.load('random_recommendations_pratik2.pt', map_location=device)

In [21]:
from sklearn.metrics import recall_score

In [22]:
recalls = []
precisions = []
for k in recommendations:
    src_nodes = recommendations[k]['src_nodes']
    scores = recommendations[k]['confidence']
    
    src_nodes = src_nodes.cpu().detach().numpy()
    scores = scores.cpu().detach().numpy()
    
    tp = 0
    total = 0
    
    for edge in edges:
        if edge[2]==1 and edge[1]==k:
            total += 1
            if edge[0] in src_nodes:
                tp += 1
    
    
    rec = 0
    pre = tp/len(src_nodes)
    precisions.append(pre)
    if total > 0:
        rec = tp/total
    recalls.append(rec)
    
print('Average recall score: ', np.mean(recalls))
print('Average precision score: ', np.mean(precisions))

Average recall score:  0.027507851734196336
Average precision score:  0.00011965401249399328


In [18]:
with torch.no_grad():
    #y = torch.tensor(test_label, dtype=torch.float)
    #edge_index = torch.tensor([test_src,test_dest], dtype=torch.long).to(device)
    test_dataset = LinkPredictionDataset(node_ids,feature,edges[train_size:])
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    batch = next(iter(test_dataloader))
    print(batch[0], batch[1])

tensor([28240]) tensor([20722])


In [19]:
np.random.seed(42)
src_nodes = np.random.randint(0, len(feature), size=50)
src_nodes = torch.from_numpy(src_nodes)

In [25]:
torch.ones(SAMPLE_SIZE, device=device, dtype=torch.int32) * 5

tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5], dtype=torch.int32)

In [28]:
SAMPLE_SIZE = 50

with torch.no_grad():
    #y = torch.tensor(test_label, dtype=torch.float)
    #edge_index = torch.tensor([test_src,test_dest], dtype=torch.long).to(device)
    test_dataset = LinkPredictionDataset(node_ids,feature,edges[train_size:])
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    recommendations = {}
    for batch_idx, batch in enumerate(test_dataloader):
        #src_nodes, dst_nodes,node_attributes,y = batch
        
#         src_nodes=batch[0].to(device)
        np.random.seed(batch_idx)
        src_nodes = np.random.randint(0, len(feature), size=SAMPLE_SIZE)
        src_nodes = torch.from_numpy(src_nodes).to(device)
#         print(dst_nodes.item())
        dst_node = batch[1].item()
        dst_nodes = torch.ones(SAMPLE_SIZE, device=device, dtype=torch.int32) * dst_node
#         dst_nodes=batch[1].to(device)
        node_attributes=batch[2].to(device)
#         y = batch[3].to(device)
        z = model.encode(node_attributes,edge_index)
        predictions=torch.round(torch.sigmoid(model.decode(z,src_nodes,dst_nodes))).view(-1)
        recommendations[dst_node] = {'src_nodes': src_nodes, 'confidence': predictions}
        #print(predictions)
#         if predictions==y.float():count+=1
#         g_truth.append(y.item())
#         prediction.append(predictions.item())
        #print('Predictions:', predictions)
# print("Accuracy:",(count/(batch_idx+1))*100,"%")
# cm = confusion_matrix(np.array(g_truth),np.array(prediction))

# #print(cm)
# labels = ['Class 0', 'Class 1']
# cmap = plt.cm.Blues

# # Create figure and axes
# fig, ax = plt.subplots()
# im = ax.imshow(cm, interpolation='nearest', cmap=cmap)

# # Show all ticks and label them with the respective list entries
# ax.set_xticks(range(len(labels)))
# ax.set_yticks(range(len(labels)))
# ax.set_xticklabels(labels)
# ax.set_yticklabels(labels)

# # Add text inside each cell
# for i in range(len(labels)):
#     for j in range(len(labels)):
#         ax.text(j, i, str(cm[i, j]), ha="center", va="center", color="white" if cm[i, j] > cm.max() / 2 else "black")

# # Set plot labels and title
# plt.xlabel("Predicted labels")
# plt.ylabel("True labels")
# plt.title("Confusion Matrix")

# # Add a color bar
# plt.colorbar(im)

# # Show the plot
# #plt.show()
# plt.savefig('scratch/research/plots/adadelta_nlp_sage_cnf_matrix.jpeg')
# # %%
# f1 = f1_score(np.array(g_truth),np.array(prediction))

# print("F1 score:", f1)

KeyboardInterrupt: 

In [ ]:
torch.save(recommendations, 'random_recommendations_pratik.pt')

In [ ]:
recommendations = {1: {'src_nodes':[0], 'confidence':[1]}

In [ ]:
for 

In [ ]:
for 
for edge in edges:
    if edge[2]==1 and edge[1]==

In [2]:
# df = pd.read_csv('/home/scai/msr/aiy227513/scratch/research/superSetAnalysis.csv')

In [3]:
# node_ids =[]
# node_features = []
# req_A=['citationVelocity', 'influentialCitationCount', 'isOpenAccess',
#        'isPublisherLicence', 'numCitebBy', 'numCiting','year']
# for index, row in df.iterrows():
#     paperId = row['paperId']
#     node_ids.append(paperId)
#     f=[]
#     for attribute in req_A:
#             if attribute=="isOpenAccess":
#              if row[attribute]==True:attribute_value=1
#              else:attribute_value=0
#             elif attribute=="isPublisherLicence":
#               if row[attribute]==True:attribute_value=1
#               else:attribute_value=0
#             else:
#                attribute_value = row[attribute]
#             f.append(attribute_value)
#     node_features.append(np.array(f).astype(np.float32))

In [ ]:
# with open("/home/scai/msr/aiy227513/scratch/research/edges_same_ratio.pickle", "rb") as file:
#     edges = pickle.load(file)
#     file.close()